In [ ]:
# !pip install numpy pandas scipy 

In [3]:
import numpy as np
from scipy.linalg import lstsq
import random
import pandas as pd
import os

type_converter = {i: lambda x: int(x, 16) for i in range(16)}
LAB_PATH = "/scratch/net4/HOS/Traces"
MAX_ROWS = 30000 #Total Number of rows to be selected
ROUND1_START = 20000
ROUND1_END = 25000
SCALE = 3
m_off = 0
AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA,0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]
random_plaintexts = np.loadtxt(os.path.join(LAB_PATH,"plaintexts.txt"), dtype=np.uint8, max_rows=MAX_ROWS, converters=type_converter)
random_ciphertexts = np.loadtxt(os.path.join(LAB_PATH,"ciphertexts.txt"), dtype=np.uint8, max_rows=MAX_ROWS, converters=type_converter)
random_traces = np.loadtxt(os.path.join(LAB_PATH,"traces.txt"), max_rows=MAX_ROWS)
unscaled_traces = random_traces[:,ROUND1_START:ROUND1_END]

scaled_traces = ((unscaled_traces / 127) * 4) * SCALE + m_off

print("unscaled traces shape: ", unscaled_traces.shape)
print("scaled traces shape: ", scaled_traces.shape)

unscaled traces shape:  (30000, 5000)
scaled traces shape:  (30000, 5000)


## Stochastic Model for DSCA 
- Constructing Basis functions
- Profiling based on SBOX output of 1st round
- Key Extraction based on minimum residual error

In [4]:
import numpy as np
from scipy.linalg import lstsq, pinv
import random

class BasisFunctions:
    """Handles the precomputation of the basis matrix G for a 4-bit S-Box."""
    def __init__(self, sbox, basis_type='bits'):
        self.sbox = sbox
        if basis_type in ["hw", "lsb"]:
            self.num_basis = 2
        else:
            self.num_basis = 5
        self.basis_type = basis_type
        self.G = self._build_basis_matrix()

    def _build_basis_matrix(self):
        """Precomputes the basis matrix G for all phi values (0-15) based on actual bit value and HW"""
        G = np.zeros((16, self.num_basis), dtype=float)
        for phi in range(16):
            sbox_output = self.sbox[phi]
            if(self.basis_type == 'bits'):
                G[phi] = [1.0]+[float(i) for i in format(sbox_output, '04b')]
            elif(self.basis_type == 'hw_bits'):
                G[phi] = [1.0] + [float(i) for i in format(bin(sbox_output).count('1'),'04b')]
            elif(self.basis_type == 'hw'):
                G[phi] = [1.0] + [float(bin(sbox_output).count('1'))]
            elif(self.basis_type == 'lsb'):
                G[phi] = [1.0, float(sbox_output & 1)]
        return G

class Profiling:
    """Manages the profiling phase: beta estimation, time point selection, and covariance estimation."""
    def __init__(self, basis_functions):
        self.basis_functions = basis_functions
        self.betas = None
        self.ts = None
        self.cov = None

    def estimate_betas(self, traces_prof, x_prof, k_b, start_time, end_time):
        phi_prof = np.bitwise_xor(x_prof, k_b) & 0x0F
        G_prof = self.basis_functions.G[phi_prof]  # shape (N1, num_basis)
        I = traces_prof[:, start_time:end_time]  # shape (N1, segment_length)
        G_pinv = pinv(G_prof)
        betas = G_pinv @ I    # Shape (num_basis, segment_length)
        self.betas = betas.T  # Match old shape (segment_length, num_basis)
        self.start_time = start_time
        return self.betas

    def select_time_points(self,betas, tau, selection_mode='S2', traces_prof=None):
        """Selects relevant time points ts based on the norm of data-dependent betas."""
        norm_b = np.linalg.norm(betas[:, 1:], axis=1)  # Exclude constant term
        if selection_mode == 'S3':
            self.ts = [np.argmax(norm_b)]  #Single max peak
        elif selection_mode == 'S2':
            self.ts = [int(x) for x in np.argsort(norm_b)[-5:]]  # Top 10 peaks
        elif selection_mode == 'S6':
            self.ts = np.argsort(norm_b)[-21:]  #Top 21 peaks
        elif selection_mode == 'S1':
            self.ts = np.where(norm_b >= tau)[0]  #Threshold-based
        elif selection_mode in ['S4', 'S5']:
            if traces_prof is None:
                raise ValueError("traces_prof required for S4/S5 selection modes")
            var_t = np.var(traces_prof, axis=0)
            mask = (norm_b >= tau) & (norm_b > var_t)
            self.ts = np.where(mask)[0]
            if selection_mode == 'S5':
                extra_mask = (norm_b >= tau / 2) & ~mask
                self.ts = np.concatenate((self.ts, np.where(extra_mask)[0]))
        else:
            raise ValueError(f"Unknown selection_mode: {selection_mode}")
        return self.ts
    
    def detect_noise(self, traces_prof, x_prof, k_b):
        """Detect noise levels using residuals after beta fitting."""
        if self.betas is None:
            raise ValueError("Run estimate_betas first")
        
        # Compute predicted h_t for all t in segment
        phi_prof = np.bitwise_xor(x_prof, k_b) & 0x0F
        G_prof = self.basis_functions.G[phi_prof]  # N1 x 5
        h_pred = np.dot(G_prof, self.betas.T)  # N1 x segment_length
        # Residuals = observed - predicted
        segment_traces = traces_prof[:, self.start_time:self.start_time + len(self.betas)]
        residuals = segment_traces - h_pred
        # Noise metrics
        noise_var = np.var(residuals)  # Overall var(R_t)
        signal_var = np.var(h_pred)    # var(h_t)
        snr = signal_var / noise_var if noise_var > 0 else float('inf')
        # Per-time variance
        noise_var_per_t = np.var(residuals, axis=0)  # segment_length x 1
        return snr, noise_var, noise_var_per_t


class KeyExtraction:
    """Handles key extraction using minimum or maximum likelihood principles."""
    def __init__(self, profiling):
        self.profiling = profiling
        self.basis_functions = profiling.basis_functions
        self.betas = profiling.betas
        self.ts = profiling.ts
        self.cov = profiling.cov

    def extract_key(self, traces_attack, x_attack,betas, ts,  N3, method='minimum'):
        """Extracts the subkey using the specified method."""
        extracted_key = {i: 0 for i in range(16)}
        G = self.basis_functions.G
        betas_ts = betas[ts]
        if method == 'minimum':
            min_diff = float('inf')
            best_k = None
            for k_prime in range(16):
                diff = 0.0
                for j in range(N3):
                    phi_j = np.bitwise_xor(x_attack[j], k_prime) & 0x0F
                    h_j = betas_ts @ G[phi_j]
                    i_j = traces_attack[j, ts]
                    diff += np.sum((i_j - h_j) ** 2)
                avg_diff = diff / N3
                if avg_diff < min_diff:
                    min_diff = avg_diff
                    best_k = k_prime
                    extracted_key[k_prime] = extracted_key[k_prime]+1
            return best_k, extracted_key
        else:
            raise ValueError(f"Unknown method: {method}")


    def compute_success_rate(self, traces_attack, x_attack,betas,timepoints, correct_k, sampled=50, num_trials=10, method='minimum'):
        """Computes the success rate over multiple trials with random subsets."""
        num_attack = len(x_attack)
        success = 0
        for _ in range(num_trials):
            indices = random.sample(range(num_attack), sampled)
            traces_sub = traces_attack[indices]
            x_sub = x_attack[indices]
            extracted_k = self.extract_key(traces_sub, x_sub,betas, timepoints, sampled, method)
            if extracted_k == correct_k:
                success += 1
        return (success / num_trials) * 100
    
def create_range(start_time=0, end_time=5000, increments=(5, 10), widths=(10, 20)):
    ranges = set()
    widths = tuple(sorted(widths))
    min_w = widths[0]

    for inc in increments:
        s = start_time
        while s + min_w <= end_time:
            for w in widths:
                end = s + w
                if end > end_time:
                    break
                ranges.add((s, end))
            s += inc
    return sorted(ranges)

## Profiling and Key Extraction using Known Key

In [5]:
# import SDSCA as sd
total_prof = 20000
key = np.array([15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0], dtype=np.uint8)
ATTACK_START = 25000
ATTACK_END = 30000
TOTAL_ATTACK = ATTACK_END - ATTACK_START
PROF_START = 0
PROF_END = 5000
correct_count = 0
top_n = 4
results = []
for byte_idx in range(16):
    plaintexts_all = random_plaintexts[:, byte_idx] & 0x0F  # Lower 4 bits
    k_b = key[byte_idx] & 0x0F
    traces_prof = scaled_traces[:total_prof]
    plaintexts_prof = plaintexts_all[:total_prof]
    AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA,0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]
    bf = BasisFunctions(AES_SBOX, basis_type="hw")
    prof = Profiling(bf)
    betas = prof.estimate_betas(traces_prof, plaintexts_prof, k_b,PROF_START, PROF_END)
    timepoints = prof.select_time_points(betas, 0.1, selection_mode="S2")
    traces_attack = scaled_traces[ATTACK_START:ATTACK_END]
    plaintexts_attack = plaintexts_all[ATTACK_START:ATTACK_END]
    ke = KeyExtraction(prof)
    extracted_k, exp_keys = ke.extract_key(traces_attack, plaintexts_attack, betas, timepoints, TOTAL_ATTACK, method="minimum")
    filtered = {k: v for k, v in exp_keys.items() if v}
    if key[byte_idx] in filtered.keys():
        correct_count += 1
    sorted_candidates = sorted(filtered.items(), key=lambda x: x[1], reverse=True)
    top_candidates = [int(k) for k, _ in sorted_candidates]

    correct_key = int(key[byte_idx])
    if correct_key==extracted_k:
        success = True
        correct_count += 1
    else:
        success = False

    # print(f"Byte {byte_idx:02d}: Extracted key = {extracted_k}, "
    #       f"Top candidates = {top_candidates}, "
    #       f"Correct key = {correct_key}, Success = {success}")

    row = {
        "byte": byte_idx,
        "correct_key": correct_key,
        "extracted_key": int(extracted_k),
        "candidates": top_candidates,
        "success": success
    }
    results.append(row)

df_attack = pd.DataFrame(results,
                         columns=["byte", "correct_key", "extracted_key",
                                  "candidates","success"])

In [6]:
df_attack

,byte,correct_key,extracted_key,candidates,success
0,0,15,15,"[0, 2, 3, 15]",True
1,1,14,14,"[0, 3, 14]",True
2,2,13,14,"[0, 1, 2, 6, 13, 14]",False
3,3,12,12,"[0, 12]",True
4,4,11,11,"[0, 1, 7, 8, 11]",True
5,5,10,10,"[0, 1, 6, 10]",True
6,6,9,9,"[0, 3, 9]",True
7,7,8,8,"[0, 1, 2, 8]",True
8,8,7,7,"[0, 4, 6, 7]",True
9,9,6,6,"[0, 4, 5, 6]",True


In [ ]:
plaintext_hex = ''.join([f'{x:x}' for x in random_plaintexts[0]])
ciphertext_hex = ''.join([f'{x:x}' for x in random_ciphertexts[0]])

('b3de3c5a10a85cc7', 'd416155d2122b635')

In [13]:
df_attack["extracted_key"].tolist()

[15, 14, 14, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

## Profiling and Key Extraction using Unknown Key

In [ ]:
try:
    from tqdm import tqdm
    use_tqdm = True
except Exception:
    use_tqdm = False

byte_iter = range(16)
if use_tqdm:
    byte_iter = tqdm(byte_iter, desc="bytes")
top_n = 5
AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA, 0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]
best_candidates_per_byte = {} 
candidate_range = range(16)

for byte_idx in byte_iter:
    plaintexts_all = random_plaintexts[:, byte_idx] & 0x0F
    traces_prof = scaled_traces[:total_prof]
    plaintexts_prof = plaintexts_all[:total_prof]

    candidate_scores = []

    for cand in candidate_range:
        k_b = int(cand & 0x0F)

        bf = BasisFunctions(AES_SBOX, basis_type="hw")
        prof = Profiling(bf)

        try:
            betas = prof.estimate_betas(traces_prof, plaintexts_prof, k_b, PROF_START, PROF_END)
        except Exception as e:
            print(f"[byte {byte_idx} cand {cand}] estimate_betas failed: {e}")
            continue
        try:
            timepoints = prof.select_time_points(betas, 0.1, selection_mode="S2")
        except Exception as e:
            print(f"[byte {byte_idx} cand {cand}] select_time_points failed: {e}")
            continue

        traces_attack = scaled_traces[ATTACK_START:ATTACK_END]
        plaintexts_attack = plaintexts_all[ATTACK_START:ATTACK_END]

        ke = KeyExtraction(prof)
        try:
            extracted_k, exp_keys = ke.extract_key(traces_attack, plaintexts_attack, betas, timepoints, TOTAL_ATTACK, method="minimum")
        except Exception as e:
            print(f"[byte {byte_idx} cand {cand}] extract_key failed: {e}")
            continue

        # Score the candidate:
        # use extracted_key.get(cand,0) as the primary score, plus a bonus if extracted_k == cand.
        score = 0.0
        try:
            if isinstance(exp_keys, dict):
                score = float(exp_keys.get(cand, 0))
            else:
                # give small score if extracted_k equals candidate
                score = 1.0 if extracted_k == cand else 0.0
        except Exception:
            score = 0.0

        # Strong bonus if the returned extracted_k equals this candidate (means this run found this nibble)
        if extracted_k == cand:
            score += 10.0
        try:
            exp_keys_snapshot = dict(exp_keys) if isinstance(exp_keys, dict) else {"value": exp_keys}
        except Exception:
            exp_keys_snapshot = {"unserializable": str(type(exp_keys))}

        candidate_scores.append((cand, score, extracted_k, exp_keys_snapshot))

    # sort and keep top_n
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    best_candidates_per_byte[byte_idx] = candidate_scores[:top_n]

bytes: 100%|██████████| 16/16 [03:49<00:00, 14.36s/it]


In [ ]:
top_k = 4
csv_filename = "byte_top4_separate_cols.csv"
all_success = True
success_count = 0
result_rows = []
for b in range(16):
    # get correct key nibble if `key` exists in scope, else None
    try:
        correct_key = int(key[b])
    except Exception:
        correct_key = None

    cand_list = best_candidates_per_byte.get(b, [])
    top_candidates = [int(t[0]) for t in cand_list[:top_k]]
    if len(top_candidates) < top_k:
        top_candidates += [None] * (top_k - len(top_candidates))

    if correct_key is None:
        success = False
    else:
        success = any((cand is not None and int(cand) == int(correct_key)) for cand in top_candidates)

    if success:
        success_count += 1
    else:
        all_success = False

    row = {
        "byte": b,
        "correct_key": correct_key,
        "candidates":top_candidates,
        "cand_1": top_candidates[0],
        "cand_2": top_candidates[1],
        "cand_3": top_candidates[2],
        "cand_4": top_candidates[3],
        "success": success
    }
    result_rows.append(row)

df_top4 = pd.DataFrame(result_rows, columns=["byte", "correct_key", "cand_1", "cand_2", "cand_3", "cand_4", "success"])
df_top4.to_csv(csv_filename, index=False)

if all_success and success_count == 16:
    print("Key recovered successfully ✅")
else:
    print(f"Key recovery incomplete: {success_count}/16 bytes matched one of the top-{top_k} candidates.")


Key recovered successfully ✅


In [10]:
df_top4

,byte,correct_key,cand_1,cand_2,cand_3,cand_4,success
0,0,15,5,11,15,0,True
1,1,14,7,8,14,0,True
2,2,13,9,0,4,13,True
3,3,12,6,12,0,1,True
4,4,11,5,11,12,0,True
5,5,10,10,14,0,2,True
6,6,9,8,9,0,1,True
7,7,8,8,12,13,0,True
8,8,7,1,2,7,8,True
9,9,6,6,0,5,1,True


In [ ]:
import ast
import csv
import pickle
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

plaintext_hex = ''.join([f'{x:x}' for x in random_plaintexts[0]])
ciphertext_hex_target = ''.join([f'{x:x}' for x in random_ciphertexts[0]])

def aes_encrypt_with_keybytes(key_bytes: bytes, plaintext_bytes: bytes) -> str:
    padded = pad(plaintext_bytes, AES.block_size)
    print("Padded Plaintext (hex):", padded.hex())
    cipher = AES.new(key_bytes, AES.MODE_ECB)
    ct = cipher.encrypt(padded)
    return ct.hex()

def parse_candidates_field(val):
    """
    Safely parse a 'candidates' cell which might already be a list or might be a string representation.
    If parsing fails, return None.
    """
    if val is None:
        return None
    if isinstance(val, (list, tuple, set)):
        return [int(x) for x in val]
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, (list, tuple, set)):
                return [int(x) for x in parsed]
        except Exception:
            pass
    return None

base_key_bytes = df_attack["extracted_key"].tolist()
if len(base_key_bytes) != 16:
    raise ValueError("Expecting `key` to contain 16 elements (one per byte).")

# ---------- find failed indices ----------
failed_rows = []
if 'df_attack' in globals() and 'success' in df_attack.columns:
    failed_rows = df_attack[df_attack['success'] == False].to_dict('records')

if not failed_rows:
    print("No failed indices found (no rows with success==False). Nothing to brute-force.")
else:
    print("Failed rows (to brute-force):", [r['byte'] for r in failed_rows])

bruteforce_results = []  # list of dicts: {byte, candidate, ciphertext, match (bool)}

for row in failed_rows:
    b = int(row.get("byte"))
    # Get candidate list from the DataFrame row if present; else try full 0..15
    raw_candidates = row.get("candidates", None)
    candidates = parse_candidates_field(raw_candidates)
    if not candidates:
        candidates = list(range(16))

    print(f"\nBrute-forcing byte index {b} with candidates: {candidates}")

    found_match_for_byte = False
    plaintext_bytes = bytes.fromhex(''.join([f'{x:02x}' for x in random_plaintexts[0]]))
    for cand in candidates:
        candidate_key = base_key_bytes       # copy base key
        candidate_key[b] = int(cand)                # replace that byte with candidate nibble
        key_bytes = bytes.fromhex(''.join([f'{x:02x}' for x in candidate_key]))            # 16-byte AES key
        ct_hex = aes_encrypt_with_keybytes(key_bytes, plaintext_bytes)
        print(ct_hex)
        is_match = False
        if ciphertext_hex_target is not None:
            if ct_hex.lower() == ciphertext_hex_target.lower():
                is_match = True
                found_match_for_byte = True
                print(f"  -> MATCH! byte {b} candidate {cand} -> key = {key_bytes.hex()} -> ciphertext = {ct_hex}")
                break
        else:
            print(f"  cand {cand:2d}: ciphertext = {ct_hex}")